<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/LogisticRegression_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm
import torch.distributions as distr

In [4]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 10.0 MB/s eta 0:00:00


In [5]:
import torchmetrics
import torchvision
from torchvision import transforms as T

Loading and preprocessing the data

In [6]:
#Load the dataset
penguins = tfds.load('penguins', as_supervised=True, split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/334 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/penguins/processed/1.0.0.incomplete3ZIN8L/penguins-train.tfrecord*...:   0…

Dataset penguins downloaded and prepared to /root/tensorflow_datasets/penguins/processed/1.0.0. Subsequent calls will reuse this data.


In [7]:
# By default, the Dataset object is an iterator over the elements.
# The instructions below extract the underlying tensors.
X, y = penguins.batch(500).get_single_element()
X, y = X.numpy(), y.numpy()

In [8]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [9]:
Xtrain = torch.from_numpy(Xtrain).float()
Xtest = torch.from_numpy(Xtest).float()

In [10]:
ytrain = torch.from_numpy(ytrain).long()
ytest = torch.from_numpy(ytest).long()

Define Logistic Regression

In [11]:
from torch import nn
from torch.nn import functional as F

In [12]:
class SimpleLogisticRegression(nn.Module):
  def __init__(self, input_size, w, b):
    super().__init__()
    self.weight = nn.Parameter(w)
    self.bias = nn.Parameter(b)


  def forward(self, x):
    x = x.reshape(1, -1)
    return torch.softmax(x@self.weight + self.bias, 1)

In [13]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


Initialize the parameters

In [14]:
# We initialize the parameters randomly and the model with an input size
w = torch.randn((4, 3), requires_grad=False)
b = torch.randn((3, ), requires_grad=False)
LG = SimpleLogisticRegression(4, w, b).to(device)

In [15]:
# Note: we also need to move data when asking for a prediction
print(LG(Xtrain[0].to(device)))

tensor([[0.2662, 0.0604, 0.6734]], grad_fn=<SoftmaxBackward0>)


Train and evaluate the network with forward gradient

In [16]:
def accuracy(ytrue, ypred):
  return (ypred.argmax(1) == ytrue).float().mean()

In [17]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(ytrain[0].to(device), LG(Xtrain[0].to(device)))

tensor(0.)

Define Cross Entropy

In [18]:
def cross_entropyW(w, ytrue, x, b):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j].reshape(1, -1)
    ypred[j]=torch.softmax(xj@w+b,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [19]:
from torch.nn import functional as F

In [20]:
def cross_entropyB(b, ytrue, x, w):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j].reshape(1, -1)
    ypred[j]=torch.softmax(xj@w+b,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [21]:
print(cross_entropyB(b, ytrain,Xtrain, w))


tensor(1.3301)


In [22]:
print(LG(Xtrain[0].to(device)))

tensor([[0.2662, 0.0604, 0.6734]], grad_fn=<SoftmaxBackward0>)


In [23]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [24]:
def rosenbrock_function(x):
  sum=0
  for i in range (x.size(0) -1):
    sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(1-x[i], 2))
  return sum

In [25]:
def sphere_function(x):
  sum=0
  for i in range(x.size(0)):
    sum += torch.pow(x[i], 2)
  return sum

In [26]:
from functorch import jvp
from functools import partial

In [27]:
def train_fwd_gradient(x, y):
  #x, y = x.to(device), y.to(device)

  l_rate0 = 0.1

  #Parameters
  w = torch.randn((4, 3))
  #w = torch.div(w, torch.norm(w, 2))

  b = torch.randn((3, ))
  #b = torch.div(b, torch.norm(b, 2))

  ftw = cross_entropyW(w, y, x, b)

  LG = SimpleLogisticRegression(4, w, b)
  #w1 = w.reshape(-1)
  #w1 = torch.cat((w1, b), 0)
  error=0
  for i in range (x.size(0)):
    if (LG(x[i]).argmax(1)- y[i])!=0:
        error = error+ 1
  error= error / x.size(0)
  t=torch.tensor([0])
  print('Iteracion', t, 'loss', ftw)

  while (ftw>0.1) :

    t=t+1
    vw=torch.randn(w.shape)
    vw = (vw - torch.mean(vw))/torch.std(vw)
    vb=torch.randn(b.shape)
    vb = (vb - torch.mean(vb))/torch.std(vb)

    fw = partial(cross_entropyW,ytrue=y, x=x, b=b)
    fb = partial(cross_entropyB,ytrue=y, x=x, w=w)

    ftw, dtw = jvp(fw,(w, ), (vw, ))
    ftb, dtb = jvp(fb,(b, ), (vb, ))
    gtw=vw*dtw
    gtb = vb*dtb
    w = w - l_rate0*gtw
    b = b - l_rate0*gtb


    #w = w1.reshape(-1, 3)
    LG = SimpleLogisticRegression(4, w, b)
    error=0
    for i in range (x.size(0)):
      if (LG(x[i]).argmax(1)- y[i])!=0:
        error = error+ 1
    error= error / x.size(0)
    print('Iteracion', t, 'loss', ftw)
  return w, b

In [28]:
w, b = train_fwd_gradient(Xtrain, ytrain)

Iteracion tensor([0]) loss tensor(1.5799)
Iteracion tensor([1]) loss tensor(1.5799)
Iteracion tensor([2]) loss tensor(1.5749)
Iteracion tensor([3]) loss tensor(1.5642)
Iteracion tensor([4]) loss tensor(1.5440)
Iteracion tensor([5]) loss tensor(1.5323)
Iteracion tensor([6]) loss tensor(1.5183)
Iteracion tensor([7]) loss tensor(1.5011)
Iteracion tensor([8]) loss tensor(1.4997)
Iteracion tensor([9]) loss tensor(1.4853)
Iteracion tensor([10]) loss tensor(1.4715)
Iteracion tensor([11]) loss tensor(1.4631)
Iteracion tensor([12]) loss tensor(1.4571)
Iteracion tensor([13]) loss tensor(1.4542)
Iteracion tensor([14]) loss tensor(1.4476)
Iteracion tensor([15]) loss tensor(1.4453)
Iteracion tensor([16]) loss tensor(1.4375)
Iteracion tensor([17]) loss tensor(1.4313)
Iteracion tensor([18]) loss tensor(1.4235)
Iteracion tensor([19]) loss tensor(1.4044)
Iteracion tensor([20]) loss tensor(1.4003)
Iteracion tensor([21]) loss tensor(1.3829)
Iteracion tensor([22]) loss tensor(1.3768)
Iteracion tensor([23]

In [30]:
LG = SimpleLogisticRegression(4, w, b)
error=0
for i in range (Xtest.size(0)):
  if (LG(Xtest[i]).argmax(1)- ytest[i])!=0:
    error = error+ 1
error= error / Xtest.size(0)


In [31]:
print(error)

0.023809523809523808
